### Import packages

In [1]:
import math
from scipy.stats import ttest_ind

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

C:\Users\Mateo P G\AppData\Local\Temp\ipykernel_22796\2563196602.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Creating functions

In [2]:
def load_data_set(data = 'train'):
    """_summary_
    This function reads the csv that contains the training data or test data for the project
    Args:
        data (str, optional): _description_. Defaults to 'train'.

    Returns:
        _type_: _description_
    """
    return pd.read_csv(f'../Data/{data}.csv')

def find_closest_cabin(fare_value, means_dataset):
    """_summary_
    This function finds the closest class in a class column using the mean value of other given column (fare in this case) and a given Fare value to compare.
    Args:
        fare_value (_type_): _description_
        means_dataset (_type_): _description_

    Returns:
        _type_: _description_
    """
    closest_letter = min(means_dataset['Fare'].keys(), key=lambda x: abs(means_dataset['Fare'][x] - fare_value))
    return closest_letter

def imput_age_by_pclass_and_sibsp(row_sibps_and_pclass, grouped_ages):
    """_summary_
    This function finds the mean/median or other metric value for the age grouped by SibSp and Pclass and imputs it into a DataFrame given a SibSp and a Pclass
    Args:
        row_sibps_and_pclass (_type_): _description_
        grouped_ages (_type_): _description_

    Returns:
        _type_: _description_
    """
    age_to_imput = grouped_ages[(grouped_ages['SibSp']== row_sibps_and_pclass['SibSp'])
                                &(grouped_ages['Pclass']== row_sibps_and_pclass['Pclass'])]['Age'].values[0]
    if not age_to_imput > 0:
        age_to_imput = grouped_ages['Age'].mean()
    return age_to_imput

### Loading data

In [3]:
original_train_set = load_data_set()
original_test_set = load_data_set(data = 'test')

### EDA

 The percentage of nulls in the columns tell us that Age, Cabin and Embarked could be inferred. The Cabin could be difficult, but as it could be highly related with the Fare, something could be made

In [4]:
original_train_set.isnull().sum() / len(original_train_set) * 100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

Boxplots between some variables and Age, with the aim of finding relations to imput Age data

In [5]:
fig = make_subplots(
    rows=3, cols=2,
    specs=[[{}, {}],
           [{}, {}],
           [{"colspan": 2}, None]],
    subplot_titles=("Sex", "Embarked", "Pclass", "SibSp", "Parch")
    )
fig.add_trace(go.Box(x=original_train_set["Sex"], y=original_train_set["Age"], name="Age by Sex", boxpoints="all"), row = 1, col = 1)
fig.add_trace(go.Box(x=original_train_set["Embarked"], y=original_train_set["Age"], name="Age by Port", boxpoints="all"), row = 1, col = 2)
fig.add_trace(go.Box(x=original_train_set["Pclass"], y=original_train_set["Age"], name="Age by Pclass", boxpoints="all"), row = 2, col = 1)
fig.add_trace(go.Box(x=original_train_set["SibSp"], y=original_train_set["Age"], name="Age by SibSp", boxpoints="all"), row = 2, col = 2)
fig.add_trace(go.Box(x=original_train_set["Parch"], y=original_train_set["Age"], name="Age by Parch", boxpoints="all"), row = 3, col = 1)
fig.update_layout(height=700, showlegend=False, title_text="Age by different variables")
fig.show()

Correlations between Age and other variables to find relations

In [6]:
original_train_set[original_train_set.dtypes[(original_train_set.dtypes =='int64')|(original_train_set.dtypes =='float64')].index].corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [7]:
fig = px.imshow(original_train_set[original_train_set.dtypes[(original_train_set.dtypes =='int64')|(original_train_set.dtypes =='float64')].index].corr())
fig.show()

Given SibSp and Pclass the more related variables to Age, some boxplots are represented including the 3 variables

In [8]:
fig = px.box(original_train_set[original_train_set['Age'].isna()], y="SibSp", points="all", color = 'Pclass')
fig.show()

In [9]:
fig = px.box(original_train_set[original_train_set['SibSp']==0], y="Age", points="all", color = 'Pclass')
fig.show()

Boxplot of Embarked and Fare to see relations

In [10]:
fig = px.box(original_train_set, x="Embarked", y="Fare", points="all")
fig.show()

In [11]:
fig = px.box(original_train_set[original_train_set['Embarked'].isna()], y="Fare", points="all")
fig.show()

Applying The ideas from the dataset to create or fill columns:
* Surnames could be extract - But they're too many, so not used until necessary
* Age could be tourned into months.
* The cabins maybe related to Tickets, Fare or others.
* The port of embarcation could be inferred?
* The age could be imputed using SibSp and Pclass.

In [12]:
train_set = original_train_set.copy()
train_set['surname'] = train_set['Name'].apply(lambda x: x.split(',')[0])
train_set['cabin_letter'] = train_set.apply(lambda x: x['Cabin'] if pd.isnull(x['Cabin']) else str(x['Cabin'])[0], axis = 1)
mean_fares_by_cabin = train_set[train_set['Cabin'].notnull()][['Fare','cabin_letter']].groupby('cabin_letter').mean().sort_values(by = 'Fare')
median_fares_by_cabin = train_set[train_set['Cabin'].notnull()][['Fare','cabin_letter']].groupby('cabin_letter').median().sort_values(by = 'Fare')
train_set['imputed_cabin_letter_by_mean'] = train_set['cabin_letter'].fillna(train_set['Fare'].apply(lambda x: find_closest_cabin(x, mean_fares_by_cabin)))
train_set['imputed_cabin_letter_by_median'] = train_set['cabin_letter'].fillna(train_set['Fare'].apply(lambda x: find_closest_cabin(x, median_fares_by_cabin)))
mean_fares_by_port = train_set[train_set['Cabin'].notnull()][['Fare','Embarked']].groupby('Embarked').mean().sort_values(by = 'Fare')
median_fares_by_port = train_set[train_set['Cabin'].notnull()][['Fare','Embarked']].groupby('Embarked').median().sort_values(by = 'Fare')
train_set['imputed_Embarked_by_mean'] = train_set['Embarked'].fillna(train_set['Fare'].apply(lambda x: find_closest_cabin(x, mean_fares_by_port)))
train_set['imputed_Embarked_by_median'] = train_set['Embarked'].fillna(train_set['Fare'].apply(lambda x: find_closest_cabin(x, median_fares_by_port)))
mean_ages_grouped = train_set[['Age', 'SibSp', 'Pclass']].groupby(['SibSp', 'Pclass']).mean().reset_index()
median_ages_grouped = train_set[['Age', 'SibSp', 'Pclass']].groupby(['SibSp', 'Pclass']).median().reset_index()
train_set['imputed_Age_by_mean'] = train_set['Age'].fillna(train_set.apply(lambda x: imput_age_by_pclass_and_sibsp(x, mean_ages_grouped), axis = 1))
train_set['imputed_Age_by_median'] = train_set['Age'].fillna(train_set.apply(lambda x: imput_age_by_pclass_and_sibsp(x, median_ages_grouped), axis = 1))
train_set['age_in_months'] = train_set['Age']*12
train_set['imputed_age_in_months_by_mean'] = train_set['imputed_Age_by_mean']*12
train_set['imputed_age_in_months_by_median'] = train_set['imputed_Age_by_median']*12

Boxplot of cabin_letter and Fare to see relations

In [13]:
fig = px.box(train_set, x="cabin_letter", y="Fare", points="all")
fig.show()

### Relations between survived and numercial columns

In [14]:
fig = make_subplots(
    rows = int(len(train_set.dtypes[(train_set.dtypes =='int64')|(train_set.dtypes =='float64')])/2) - 1, cols=2,
    subplot_titles = ["Survived by "+col for col in train_set.dtypes[(train_set.dtypes =='int64')|(train_set.dtypes =='float64')].index if col not in ['PassengerId', 'Survived']])
height_per_row = 200
for idx, column in enumerate(train_set.dtypes[(train_set.dtypes =='int64')|(train_set.dtypes =='float64')].index):
    if column not in ['PassengerId', 'Survived']: 
        fig.add_trace(go.Box(x=train_set["Survived"], y=train_set[column], boxpoints="all"), row = math.floor(idx/2), col = idx%2 + 1)
fig.update_layout(height=height_per_row*(math.floor(idx/2) + 1), showlegend=False, title_text="Survived relations with numerical features")
fig.show()

In [15]:
fig = make_subplots(
    rows = math.ceil(len(train_set.dtypes[(train_set.dtypes =='object')].index)/2)-2, cols=2,
    subplot_titles = ["Survived by "+ col for col in train_set.dtypes[(train_set.dtypes =='object')].index if col not in ['Name', 'Ticket', 'surname', 'Cabin']])
height_per_row = 200
idx = 0
for column in train_set.dtypes[(train_set.dtypes =='object')].index:
    grouped_train_set = train_set[['PassengerId', 'Survived', column]].groupby(['Survived', column]).count().reset_index()
    if column not in ['Name', 'Ticket', 'surname', 'Cabin']:
        survived_class = grouped_train_set['Survived'].unique()
        fig.add_trace(go.Bar(x=grouped_train_set[grouped_train_set['Survived']==survived_class[0]][column],
                             y=grouped_train_set[grouped_train_set['Survived']==survived_class[0]]['PassengerId'], 
                             name='Not Survived',
                             marker_color='Red',  
                             legendgroup = idx+1), row = math.floor(idx/2)+1, col = idx%2 + 1)
        fig.add_trace(go.Bar(x=grouped_train_set[grouped_train_set['Survived']==survived_class[1]][column],
                             y=grouped_train_set[grouped_train_set['Survived']==survived_class[1]]['PassengerId'],
                             name='Survived', 
                             marker_color='Blue',
                             legendgroup = idx+1), row = math.floor(idx/2)+1, col = idx%2 + 1)
        idx += 1
fig.update_layout(barmode='group', height=height_per_row*(math.floor(idx/2) + 1), title_text="Survived relations with categorical features", legend_tracegroupgap = height_per_row*(math.floor(idx/2) + 1))
fig.show()

### Numerical variables description after cleaning

In [16]:
train_set.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,imputed_Age_by_mean,imputed_Age_by_median,age_in_months,imputed_age_in_months_by_mean,imputed_age_in_months_by_median
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,29.518326,29.247946,356.389412,354.219909,350.975354
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,13.246754,13.261699,174.317968,158.961045,159.140388
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.420000,0.420000,5.040000,5.040000,5.040000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400,22.000000,22.000000,241.500000,264.000000,264.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,27.630201,26.000000,336.000000,331.562410,312.000000
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000,37.000000,37.000000,456.000000,444.000000,444.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,80.000000,80.000000,960.000000,960.000000,960.000000


### Finding statistical differences between numerical variables

In [17]:
list_of_columns_to_compare = ['Fare', 'imputed_Age_by_mean', 'imputed_Age_by_median', 'imputed_age_in_months_by_mean', 'imputed_age_in_months_by_median']
for column_name in list_of_columns_to_compare:
    test_result = ttest_ind(train_set[train_set['Survived'] == 1][column_name], train_set[train_set['Survived'] == 0][column_name])
    if test_result.pvalue < 0.05:
        conclusion = f"there is difference for the mean {column_name} between those who survived and those who did not in the Titanic tragedy"
    else:
        conclusion = f"there is NO difference for the mean {column_name} between those who survived and those who did not in the Titanic tragedy"
        
    print(f"""The t-test two-sided test for the variable {column_name} to find differences between the survivors gave the following results:
           statistic: {test_result.statistic:.2f}
           pvalue: {test_result.pvalue:.4f}
           Given those results, the test allows to conclude that {conclusion}
           """ )

The t-test two-sided test for the variable Fare to find differences between the survivors gave the following results:
           statistic: 7.94
           pvalue: 0.0000
           Given those results, the test allows to conclude that there is difference for the mean Fare between those who survived and those who did not in the Titanic tragedy
           
The t-test two-sided test for the variable imputed_Age_by_mean to find differences between the survivors gave the following results:
           statistic: -1.63
           pvalue: 0.1042
           Given those results, the test allows to conclude that there is NO difference for the mean imputed_Age_by_mean between those who survived and those who did not in the Titanic tragedy
           
The t-test two-sided test for the variable imputed_Age_by_median to find differences between the survivors gave the following results:
           statistic: -1.44
           pvalue: 0.1501
           Given those results, the test allows to conclude t

### Creating new features throught feature engineering
### One hot encoding

In [22]:
columns_to_onehot = ['Pclass', 'Sex', 'imputed_cabin_letter_by_mean', 'imputed_cabin_letter_by_median', 'imputed_Embarked_by_mean', 'imputed_Embarked_by_median']
one_hot_db = pd.DataFrame()

for column in columns_to_onehot:
    temp_one_hot_db = pd.get_dummies(train_set[column], prefix= column + '_')
    one_hot_db = pd.concat([one_hot_db, temp_one_hot_db], axis = 1)
    
train_set = pd.concat([train_set, one_hot_db], axis = 1)

### Combining features

In [23]:
train_set['family_members'] = train_set['SibSp'] + train_set['Parch']

In [24]:
train_set.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'surname',
       'cabin_letter', 'imputed_cabin_letter_by_mean',
       'imputed_cabin_letter_by_median', 'imputed_Embarked_by_mean',
       'imputed_Embarked_by_median', 'imputed_Age_by_mean',
       'imputed_Age_by_median', 'age_in_months',
       'imputed_age_in_months_by_mean', 'imputed_age_in_months_by_median',
       'family_members', 'Pclass__1', 'Pclass__2', 'Pclass__3', 'Sex__female',
       'Sex__male', 'imputed_cabin_letter_by_mean__A',
       'imputed_cabin_letter_by_mean__B', 'imputed_cabin_letter_by_mean__C',
       'imputed_cabin_letter_by_mean__D', 'imputed_cabin_letter_by_mean__E',
       'imputed_cabin_letter_by_mean__F', 'imputed_cabin_letter_by_mean__G',
       'imputed_cabin_letter_by_mean__T', 'imputed_cabin_letter_by_median__A',
       'imputed_cabin_letter_by_median__B',
       'imputed_cabin_letter_by_median__C',
       'imputed_cab